A few days ago my friend Cooper asked me what I thought was the expected outcome for the number of cards being in the right position in a shuffled standard deck of cards. For example, if only one card was in the right place, the deck would have a value of one. He had written an algorithm that determine this value experimentally.

To solve the problem, I tried to find the number of combinations that had a value of n. This meant the some combination of n cards were in the right spot, and all other cards were in the wrong spot. The key to using this method was correctly calculating the number of ways to arrange the remaining cards such that they are all in the wrong place. At first I thought that the only way all cards could be out of order was by rotating the correct order but I realized this was not the case for sequences longer than 3.

At this point, it dawned on me that the number of completely out of order sequences is the number of total sequences minus the  number of sequences with some positive number of elements in the correct spot. For example, the number of 15 element sequences that are completely out of order is equal to the total number of 15 element sequences (15!) minus the number of sequences with 1 element in the right spot, 2 elements in the right spot and so on.

Continuing with the example of 15, we see that there are 15 choose 1 ways to pick 1 correct element. This means that the remaining 14 elements MUST all be out of place. By ignoring the one element in the correct spot, we see that we just need to know how many 14 element sequences there are where no elements are in the right spot. We don't have a method for finding this number quite yet but we shall return. We will then look at the next case. With 2 spots, there are 15 choose 2 ways to pick 2 spots to fix and the other 13 must all be in the wrong place as with the 1 spot case. Again we need to find the number of sequences with 13 spots completely out of order. It is clear that for any j number of spots, we can fix j elements in 15 choose j ways and we need to find the number of 15-j element sequences that are completely out of order.

Looking at the pattern we see that the number of completely incorrect sequences of n elements can be derived from the number of completely incorrect sequences for each number of elements less than n. In other words, the number of completely incorrect sequences is recursive.

# Permutations and Combinations

While trying to answer this problem, I started looking in permutations and combinations. At first, I used the shuffle function from the random library to generate possible sequences. Each new sequence was added to a master list. The random generation process would continue until n! unique sequences of n elements were created. While considering the above problem, I realized this can also be done using recursion to ensure that all n! sequences are found systematically. By choosing one element as the first value, we are left with n-1 elements. There are (n-1)! ways to order these elements. We can use the previously found permutations for n-1 elements by utilizing arrays as indices. It should be noted that there are n such elements which can be assigned the first slot, therefore there are n times (n-1)! or n! permutations. I defined the function such that if an integer x where passed in, the program would find permutations for all integers less than n if they were not already available. Once this was done, it would be possible to find all permutations of length x.


In [192]:
import numpy as np
from random import shuffle
from numba import njit
from collections import defaultdict

perm_dict = defaultdict(list)

def perm(x):
    if x==1:
        perm_dict[1] = [0]
    else:
        for i in range(1,x):
            if len(perm_dict[i])==0: perm(i)
        if len(perm_dict[x])==0:
            for first_val in range(x):
                rest = np.delete(np.arange(x),first_val)
                #creates an array which is missing a certain value
                for j in perm_dict[x-1]:
                    new_entry = np.arange(x)
                    new_entry[0] = first_val
                    #the first value of this permutation is based on the above for loop
                    new_entry[1:] = rest[j]
                    #the remaining x-1 values are ordered based on a permutation of the sequence 0,1...x-2 which has x-1 elements
                    perm_dict[x].append(new_entry)            

I also wrote a fairly simple factorial and combination function. It should be noted that I had to cancel common terms in the numerator and denominator and use as few terms as possible in my combinations formula because of rounding at higher valued factorials.

In [193]:
def factorial(x):
    result = 1
    for i in range(1,x+1):
        result*=i
    return result

def comb(n,x):
    result = 1
    for i in range(x-n+1,x+1):
        result*=i
    result/=factorial(n)
    return result

# The Problem

The function below calculates the true value of the expected outcome by shuffling and counting the number of cards in the right place for a given number of cards and a given number of iterations. The values are then averaged.

In [176]:
@njit
def true(x,iter):
    full = np.arange(x)+1
    sorted_full = np.arange(x)+1
    sum = 0
    for i in range(iter):
        shuffle(full)
        sum+=np.sum(full==sorted_full)
    sum/=iter
    return sum

As we noted before, the key to solving this problem is recursively finding the number of completely incorrect sequences of a length x. We do this by counting all of the sequences that have at least one card in the right spot and subtracting this number from the total number of sequences.

In [178]:
zero_count = list()

def zerofind(x):
    if len(zero_count)==0: zero_count.append(0)
    result = 0
    while len(zero_count)<x-1:
        zerofind(len(zero_count)+1)
    if len(zero_count)>=x:
        return zero_count[x-1]
    for i in range(1,x):
        result+=comb(i,x)*zero_count[x-i-1]
        #determines the number of ways to fix i cards in the correct spot and uses the previous terms to find the number of completely incorrect sequences of length i
    result+=1
        #this is the original sequence where all cards are in the correct space; I did it this way to more easily utilize the previous values stored in the list zero_count
    result = int(factorial(x)-result)
        #subtracts the number of sequences with some correct elements from the total number
    zero_count.append(result)
    return result

The below function simply looks at all permutations of length x and returns the number of completely incorrect sequences by checking the permutation and the ordered sequence element wise. Despite vectorization with numpy, the function is still quite slow as all permutations of length x must also be found first.

To mirror the recursive zerofind function, alt_zerofind finds all values alt_zerofind(i) for i<x before finding alt_zerofind(x) by default, but this can be changed.

We see that the exhaustive counting method lines up with the recursive algorithm we found above.

In [179]:
alt_zero_count = list()

def alt_zerofind(x, single=0):
    if not single:
        #this is the default method which finds and stores all vallues of alt_zerofind(i) for i<x
        if len(alt_zero_count)==0: alt_zero_count.append(0)
        while len(alt_zero_count)<x-1:
            alt_zerofind(len(alt_zero_count)+1)
        if len(alt_zero_count)>=x:
            return alt_zero_count[x-1]
        if x not in perm_dict.keys(): perm(x)
        n = np.sum((np.mean(np.array(perm_dict[x])==np.arange(x), axis=1)==np.zeros(len(perm_dict[x]),)))
        #compares each permutation to the original sequence and counts the number of completely incorrect permutations
        alt_zero_count.append(n)
        return n
    else:
        #this method does not find previous values or store the newly calculated value; the algorithm is the same
        if len(alt_zero_count)>=x:
            return alt_zero_count[x-1]
        if x not in perm_dict.keys(): perm(x)
        n = np.sum((np.mean(np.array(perm_dict[x])==np.arange(x), axis=1)==np.zeros(len(perm_dict[x]),)))
        return n

In [211]:
for i in range(1,11):
    print(zerofind(i),alt_zerofind(i))

0 0
1 1
2 2
9 9
44 44
265 265
1854 1854
14833 14833
133496 133496
1334961 1334961


The following function calculates the expected outcome of the number of elements in the correct spot for a sequence of length x.

In [181]:
def exp(x):
    sum = 0
    if len(zero_count)<x: zerofind(x)
    for i in range(1,x):
        sum+=comb(i,x)*i*zero_count[x-i-1]
    sum+=x
    sum/=factorial(x)
    return sum

We see in the function below that the expected outcome lines up with the experimental values.

In [190]:
for i in range(2,60):
    print(exp(i),true(i,i*100000))

1.0 1.00283
1.0 1.0011933333333334
1.0 1.00104
1.0 1.000016
1.0 1.0010133333333333
1.0 0.99963
1.0 1.0003425
1.0 0.99991
1.0 0.999751
1.0 0.9986818181818182
1.0 0.9993308333333333
1.0 0.9990215384615385
1.0 1.0008714285714286
1.0 1.0015326666666666
1.0 0.99949125
1.0 1.0002188235294118
1.0 0.9999755555555555
1.0000000000000002 1.0004936842105263
1.0000000000000002 0.9993635
0.9999999999999999 0.9996561904761905
0.9999999999999998 1.0002386363636364
1.0 1.000125652173913
1.0 0.9999720833333333
0.9999999999999999 0.9993532
0.9999999999999998 0.9993273076923077
0.9999999999999998 0.9989018518518519
1.0000000000000002 1.0008410714285714
1.0 1.0002420689655172
0.9999999999999996 0.9991203333333334
1.0 0.9999819354838709
1.0000000000000002 1.000665625
1.0 1.0001745454545454
1.0000000000000002 1.0005911764705882
0.9999999999999998 1.0002202857142857
1.0 0.9992141666666666
0.9999999999999998 0.9991183783783784
0.9999999999999997 1.000118157894737
0.9999999999999997 0.9995784615384615
0.9999999

It is quite clear that all of the calculated and experimentally obtained values are fairly close to 1. I have not looked at formally proving the expected outcome will always be 1 yet. The slight variations for the algorithm's expected values are likely due to truncated digits for calculations with very large numbers.

# The Answer

In [230]:
1.0

1.0

Thanks for reading! If you notice any issues or have any questions, please feel free to reach out to me at cjj9000@gmail.com.